Making prediction based on 1 lag and 2 lags TF models

In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import os
import scipy.stats as ss
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool, sum_models, to_classifier
from sklearn.model_selection import KFold

import implicit

In [ ]:
#df = pd.read_csv("../input/feature-creating-v2/train_3lags_v3.csv", low_memory=False)

In [ ]:
#df = df.loc[df['order_count'] == 2]

In [ ]:
#threshold = 0.0004
#counts = df['service_title'].value_counts(normalize=True)
#df2 = df.loc[df['service_title'].isin(counts[counts > threshold].index), :]

In [ ]:
#d_134 = df2.loc[df2['service_title'] == 134][:121500]

In [ ]:
#d_98 = df2.loc[df2['service_title'] == 98][:121500]

In [ ]:
#df2 = df2.loc[df2['service_title'] != 134]
#df2 = df2.loc[df2['service_title'] != 98]

In [ ]:
#df2 = pd.concat([df2,d_134], axis=0)
#df2 = pd.concat([df2,d_98], axis=0)

In [ ]:
#df2.to_csv('train_3lags_semibalanced.csv', index=False)

In [ ]:
df2 = pd.read_csv("../input/irkutsk/train_3lags_semibalanced.csv", low_memory=False)

In [ ]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [ ]:
df_test = df2[['service_title', 'order_count']]

In [ ]:
df2.drop(['service',
'service_title',
'mfc',
'internal_status',
'external_status',
'order_type',
'department_id',
'custom_service_id',
'service_level',
'is_subdep',
'is_csid',
'proc_time',
'dayofweek',
'day_part',
'person',
'sole',
'legal',
'auto_ping_queue',
'win_count',
'month',
'week',
'year'], axis=1 , inplace=True)

In [ ]:
df_train = df2.loc[df2['order_count'] > 1]
df_test = df_test.loc[df_test['order_count'] > 1]
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)
df_train.drop(['index'], axis=1, inplace=True)
df_test.drop(['index'], axis=1, inplace=True)

In [ ]:
categorical = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1',
       'person_1', 'sole_1', 'legal_1', 'auto_ping_queue_1',
       'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'order_type_2', 'department_id_2',
       'custom_service_id_2', 'service_level_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2', 'year_2',
       'legal_2', 'auto_ping_queue_2','service_3',
       'service_title_3', 'mfc_3', 'internal_status_3', 'external_status_3',
       'order_type_3', 'department_id_3', 'custom_service_id_3',
       'service_level_3', 'is_subdep_3', 'is_csid_3', 'month_3', 'week_3', 'year_3',
       'dayofweek_3', 'day_part_3', 'person_3', 'sole_3', 'legal_3',
       'auto_ping_queue_3',
       'requester_type', 'gender']

In [ ]:
cat = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1',
       'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'department_id_2', 'month_2', 'week_2', 'year_2',
       'custom_service_id_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'service_3',
       'service_title_3', 'mfc_3', 'internal_status_3', 'external_status_3',
       'department_id_3', 'custom_service_id_3',
       'is_subdep_3', 'is_csid_3', 'month_3', 'week_3', 'year_3',
       'dayofweek_3', 
       'requester_type', 'gender']

In [ ]:
X = df_train.drop(['requester'], axis=1)
y = df_test['service_title']

In [ ]:
X.drop(['service_2',
'service_title_2',
'mfc_2',
'internal_status_2',
'external_status_2',
'order_type_2',
'department_id_2',
'custom_service_id_2',
'service_level_2',
'is_subdep_2',
'is_csid_2',
'proc_time_2',
'dayofweek_2',
'day_part_2',
'person_2',
'sole_2',
'legal_2',
'auto_ping_queue_2',
'win_count_2',
'month_2',
'week_2',
'year_2'], axis=1 , inplace=True)

X.drop(['service_3',
'service_title_3',
'mfc_3',
'internal_status_3',
'external_status_3',
'order_type_3',
'department_id_3',
'custom_service_id_3',
'service_level_3',
'is_subdep_3',
'is_csid_3',
'proc_time_3',
'dayofweek_3',
'day_part_3',
'person_3',
'sole_3',
'legal_3',
'auto_ping_queue_3',
'win_count_3',
'month_3',
'week_3',
'year_3'], axis=1 , inplace=True)

In [ ]:
categorical = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1',
       'person_1', 'sole_1', 'legal_1', 'auto_ping_queue_1',
       'requester_type', 'gender']

cat = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 
       'requester_type', 'gender']

In [ ]:
X[cat] = X[cat].astype('Int64')
X[cat] = X[cat].astype('object')

In [ ]:
X[categorical] = X[categorical].fillna('NA')

In [ ]:
def reduce_mem_usage(df):
    """ 
    iterate through all the columns of a dataframe and 
    modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}' 
                     'MB').format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max <\
                  np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max <\
                   np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max <\
                   np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max <\
                   np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max <\
                   np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max <\
                   np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}' 
                              'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) 
                                             / start_mem))
    
    return df

In [ ]:
X = reduce_mem_usage(X)

In [ ]:
X = X[categorical]

In [ ]:
X['person_1'] = X['person_1'].astype('int32')
X['sole_1'] = X['sole_1'].astype('int32')
X['legal_1'] = X['legal_1'].astype('int32')
X['auto_ping_queue_1'] = X['auto_ping_queue_1'].astype('int32')

In [ ]:
from sklearn import preprocessing
labeling = []
for col in X.columns:
    d = pd.DataFrame()
    d[col] = X[col].unique()
    le = preprocessing.LabelEncoder()
    le.fit(d[col])
    d[col+'_l'] = le.transform(d[col])
    d = d.sort_values(by=[col+'_l']).reset_index(drop=True)
    labeling.append(d)

In [ ]:
d = pd.DataFrame()
d['service_title'] = y.unique()
le = preprocessing.LabelEncoder()
le.fit(d['service_title'])
d['service_title'+'_l'] = le.transform(d['service_title'])
d = d.sort_values(by=['service_title'+'_l']).reset_index(drop=True)
labeling_y = d

In [ ]:
labeling[0]

In [ ]:
i = 0
for col in X.columns:
    X[col] = X[col].map(labeling[i].set_index(col).to_dict()[col+'_l'])
    i += 1

In [ ]:
y = y.map(labeling_y.set_index('service_title').to_dict()['service_title'+'_l'])

In [ ]:
weights_l = labeling_y[:]
weights_l['weights'] = 1

weights = pd.DataFrame(y)
weights = pd.merge(weights, weights_l, how='left', left_on='service_title', right_on='service_title_l')
weights['weights'].loc[weights['service_title_y'] == 4] = 0.4
weights['weights'].loc[weights['service_title_y'] == 603] = 0.6
weights['weights'].loc[weights['service_title_y'] == 98] = 0.9
weights['weights'].loc[weights['service_title_y'] == 134] = 0.87
weights = np.array(weights['weights'])

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 64 * strategy.num_replicas_in_sync
LEARNING_RATE = 1e-3 * strategy.num_replicas_in_sync 
EPOCHS = 5

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [ ]:
def build_model():
    service_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_1')
    service_title_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_title_1')
    mfc_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='mfc_1')
    internal_status_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='internal_status_1')
    external_status_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='external_status_1')
    order_type_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='order_type_1')
    department_id_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='department_id_1')
    custom_service_id_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='custom_service_id_1')
    service_level_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_level_1')
    is_subdep_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_subdep_1')
    is_csid_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_csid_1')
    dayofweek_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='dayofweek_1')
    day_part_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='day_part_1')
    month_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='month_1')
    week_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='week_1')
    year_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='year_1')
    person_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='person_1')
    sole_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='sole_1')
    legal_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='legal_1')
    auto_ping_queue_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='auto_ping_queue_1')
    requester_type = tf.keras.layers.Input((1,), dtype=tf.int32, name='requester_type')
    gender = tf.keras.layers.Input((1,), dtype=tf.int32, name='gender')
    
    service_1_embedding = tf.keras.layers.Embedding(len(X['service_1'].unique()), 11, input_length=1, name='service_1_embedding')(service_1)
    service_title_1_embedding = tf.keras.layers.Embedding(len(X['service_title_1'].unique()), 50, input_length=1, name='service_title_1_embedding')(service_title_1)
    mfc_1_embedding = tf.keras.layers.Embedding(len(X['mfc_1'].unique()), 50, input_length=1, name='mfc_1_embedding')(mfc_1)
    internal_status_1_embedding = tf.keras.layers.Embedding(len(X['internal_status_1'].unique()), 5, input_length=1, name='internal_status_1_embedding')(internal_status_1)
    external_status_1_embedding = tf.keras.layers.Embedding(len(X['external_status_1'].unique()), 14, input_length=1, name='external_status_1_embedding')(external_status_1)
    order_type_1_embedding = tf.keras.layers.Embedding(len(X['order_type_1'].unique()), 2, input_length=1, name='order_type_1_embedding')(order_type_1)
    department_id_1_embedding = tf.keras.layers.Embedding(len(X['department_id_1'].unique()), 46, input_length=1, name='department_id_1_embedding')(department_id_1)
    custom_service_id_1_embedding = tf.keras.layers.Embedding(len(X['custom_service_id_1'].unique()), 26, input_length=1, name='custom_service_id_1_embedding')(custom_service_id_1)
    service_level_1_embedding = tf.keras.layers.Embedding(len(X['service_level_1'].unique()), 3, input_length=1, name='service_level_1_embedding')(service_level_1)
    is_subdep_1_embedding = tf.keras.layers.Embedding(len(X['is_subdep_1'].unique()), 1, input_length=1, name='is_subdep_1_embedding')(is_subdep_1)
    is_csid_1_embedding = tf.keras.layers.Embedding(len(X['is_csid_1'].unique()), 1, input_length=1, name='is_csid_1_embedding')(is_csid_1)
    dayofweek_1_embedding = tf.keras.layers.Embedding(len(X['dayofweek_1'].unique()), 4, input_length=1, name='dayofweek_1_embedding')(dayofweek_1)
    day_part_1_embedding = tf.keras.layers.Embedding(len(X['day_part_1'].unique()), 2, input_length=1, name='day_part_1_embedding')(day_part_1)
    month_1_embedding = tf.keras.layers.Embedding(len(X['month_1'].unique()), 6, input_length=1, name='month_1_embedding')(month_1)
    week_1_embedding = tf.keras.layers.Embedding(len(X['week_1'].unique()), 26, input_length=1, name='week_1_embedding')(week_1)
    year_1_embedding = tf.keras.layers.Embedding(len(X['year_1'].unique()), 1, input_length=1, name='year_1_embedding')(year_1)
    person_1_embedding = tf.keras.layers.Embedding(len(X['person_1'].unique()), 1, input_length=1, name='person_1_embedding')(person_1)
    sole_1_embedding = tf.keras.layers.Embedding(len(X['sole_1'].unique()), 1, input_length=1, name='sole_1_embedding')(sole_1)
    legal_1_embedding = tf.keras.layers.Embedding(len(X['legal_1'].unique()), 1, input_length=1, name='legal_1_embedding')(legal_1)
    auto_ping_queue_1_embedding = tf.keras.layers.Embedding(len(X['auto_ping_queue_1'].unique()), 1, input_length=1, name='auto_ping_queue_1_embedding')(auto_ping_queue_1)
    requester_type_embedding = tf.keras.layers.Embedding(len(X['requester_type'].unique()), 2, input_length=1, name='requester_type_embedding')(requester_type)
    gender_embedding = tf.keras.layers.Embedding(len(X['gender'].unique()), 1, input_length=1, name='gender_embedding')(gender)

    concatenated = tf.keras.layers.Concatenate()([service_1_embedding, 
                                                  service_title_1_embedding,
                                                 mfc_1_embedding,
                                                 internal_status_1_embedding,
                                                 external_status_1_embedding,
                                                 order_type_1_embedding,
                                                 department_id_1_embedding,
                                                 custom_service_id_1_embedding,
                                                 service_level_1_embedding,
                                                 is_subdep_1_embedding,
                                                 is_csid_1_embedding,
                                                 dayofweek_1_embedding,
                                                 day_part_1_embedding,
                                                 month_1_embedding,
                                                 year_1_embedding,
                                                 person_1_embedding,
                                                 sole_1_embedding,
                                                 legal_1_embedding,
                                                 auto_ping_queue_1_embedding,
                                                 requester_type_embedding,
                                                 gender_embedding])

    conv_0 = tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu')(concatenated)
    conv_1 = tf.keras.layers.Conv1D(128, 2, padding='same', activation='relu')(concatenated)
    conv_2 = tf.keras.layers.Conv1D(128, 6, padding='same', activation='relu')(concatenated)
    conv_0 = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(conv_0)
    conv_1 = tf.keras.layers.Conv1D(64, 2, padding='same', activation='relu')(conv_1)
    conv_2 = tf.keras.layers.Conv1D(64, 6, padding='same', activation='relu')(conv_2)
    concatenated_tensor = tf.keras.layers.Concatenate(axis=1)([conv_0, conv_1, conv_2])
    out = tf.keras.layers.Flatten()(concatenated_tensor)
    
    out = tf.keras.layers.Dense(len(y.unique()), activation='softmax', name='service_title')(out)

    model = tf.keras.models.Model(inputs=[service_1, 
                                          service_title_1, 
                                          mfc_1,
                                         internal_status_1,
                                         external_status_1,
                                         order_type_1,
                                         department_id_1,
                                         custom_service_id_1,
                                         service_level_1,
                                         is_subdep_1,
                                         is_csid_1,
                                         dayofweek_1,
                                         day_part_1,
                                         month_1,
                                         week_1,
                                         year_1,
                                         person_1,
                                         sole_1,
                                         legal_1,
                                         auto_ping_queue_1,
                                         requester_type,
                                         gender], 
                                  outputs=out)

    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

    return model

In [ ]:
inputs = [np.array(X_test['service_1']),
np.array(X_test['service_title_1']),
np.array(X_test['mfc_1']),
np.array(X_test['internal_status_1']),
np.array(X_test['external_status_1']),
np.array(X_test['order_type_1']),
np.array(X_test['department_id_1']),
np.array(X_test['custom_service_id_1']),
np.array(X_test['service_level_1']),
np.array(X_test['is_subdep_1']),
np.array(X_test['is_csid_1']),
np.array(X_test['dayofweek_1']),
np.array(X_test['day_part_1']),
np.array(X_test['month_1']),
np.array(X_test['week_1']),
np.array(X_test['year_1']),
np.array(X_test['person_1']),
np.array(X_test['sole_1']),
np.array(X_test['legal_1']),
np.array(X_test['auto_ping_queue_1']),
np.array(X_test['requester_type']),
np.array(X_test['gender'])]

In [ ]:
kfold_preds = np.zeros((X_test.shape[0], len(y.unique())))
n_splits = 5
for i in range(5):
    print('#'*25)
    print('### MODEL %i'%(i+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
    model.load_weights('../input/v55fold/v5-learnedemb-%i.h5'%i)

    print('Predicting Test...')
    preds = model.predict(inputs,verbose=True)
    kfold_preds += preds/n_splits

In [ ]:
y_pred = []
for k in range(y_test.shape[0]):
    a = np.argmax(kfold_preds[k,])
    y_pred.append(a)

In [ ]:
accuracy_score(y_pred, y_test)

In [ ]:
df = pd.read_csv("../input/feature-creating/train_3lags_v1.csv", low_memory=False)

In [ ]:
sab = pd.read_csv("../input/irkutsk/sample_submission.csv")

sab_pred = sab[['requester']]

sab_pred = pd.merge(sab_pred, df, how='left', on='requester')

sab_pred.drop(['service_3',
'service_title_3',
'mfc_3',
'internal_status_3',
'external_status_3',
'order_type_3',
'department_id_3',
'custom_service_id_3',
'service_level_3',
'is_subdep_3',
'is_csid_3',
'proc_time_3',
'dayofweek_3',
'day_part_3',
'person_3',
'sole_3',
'legal_3',
'auto_ping_queue_3',
'win_count_3', 'year_3', 'month_3', 'week_3'], axis=1 , inplace=True)

sab_pred.drop(['requester'], axis=1, inplace=True)

In [ ]:
idx = sab_pred.loc[sab_pred['order_count'] == 1].index

In [ ]:
sab_pred = sab_pred.iloc[idx]

In [ ]:
sab_pred.drop(['service_2',
'service_title_2',
'mfc_2',
'internal_status_2',
'external_status_2',
'order_type_2',
'department_id_2',
'custom_service_id_2',
'service_level_2',
'is_subdep_2',
'is_csid_2',
'proc_time_2',
'dayofweek_2',
'day_part_2',
'person_2',
'sole_2',
'legal_2',
'auto_ping_queue_2',
'win_count_2',
'month_2',
'week_2',
'year_2'], axis=1 , inplace=True)

sab_pred.drop(['service_1',
'service_title_1',
'mfc_1',
'internal_status_1',
'external_status_1',
'order_type_1',
'department_id_1',
'custom_service_id_1',
'service_level_1',
'is_subdep_1',
'is_csid_1',
'proc_time_1',
'dayofweek_1',
'day_part_1',
'person_1',
'sole_1',
'legal_1',
'auto_ping_queue_1',
'win_count_1',
'month_1',
'week_1',
'year_1'], axis=1 , inplace=True)

In [ ]:
sab_pred.drop(['proc_time', 'win_count', 'age', 'order_count'], axis=1, inplace=True)

In [ ]:
new_cols=X.columns
sab_pred.rename(columns=dict(zip(sab_pred.columns, new_cols)),inplace=True)
sab_pred[cat] = sab_pred[cat].astype('Int64')
sab_pred[cat] = sab_pred[cat].astype('object')

In [ ]:
sab_pred['person_1'] = sab_pred['person_1'].map({True: 1, False:0})
sab_pred['sole_1'] = sab_pred['sole_1'].map({True: 1, False:0})
sab_pred['legal_1'] = sab_pred['legal_1'].map({True: 1, False:0})
sab_pred['auto_ping_queue_1'] = sab_pred['auto_ping_queue_1'].map({True: 1, False:0})

In [ ]:
i = 0
for col in sab_pred.columns:
    sab_pred[col] = sab_pred[col].map(labeling[i].set_index(col).to_dict()[col+'_l'])
    i += 1

In [ ]:
sab_pred.fillna(0, inplace=True)

In [ ]:
toint = ['service_1', 'service_title_1', 'mfc_1', 'order_type_1', 'person_1', 'sole_1', 'auto_ping_queue_1', 'day_part_1',
       'internal_status_1', 'external_status_1', 'service_level_1', 'legal_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 
       'requester_type', 'gender']

In [ ]:
sab_pred[toint] = sab_pred[toint].astype('int64')

In [ ]:
sab_pred['order_type_1'] = sab_pred['order_type_1'].astype('category')
sab_pred['service_level_1'] = sab_pred['service_level_1'].astype('category')
sab_pred['day_part_1'] = sab_pred['day_part_1'].astype('category')

In [ ]:
inputs_t = [np.array(sab_pred['service_1']),
np.array(sab_pred['service_title_1']),
np.array(sab_pred['mfc_1']),
np.array(sab_pred['internal_status_1']),
np.array(sab_pred['external_status_1']),
np.array(sab_pred['order_type_1']),
np.array(sab_pred['department_id_1']),
np.array(sab_pred['custom_service_id_1']),
np.array(sab_pred['service_level_1']),
np.array(sab_pred['is_subdep_1']),
np.array(sab_pred['is_csid_1']),
np.array(sab_pred['dayofweek_1']),
np.array(sab_pred['day_part_1']),
np.array(sab_pred['month_1']),
np.array(sab_pred['week_1']),
np.array(sab_pred['year_1']),
np.array(sab_pred['person_1']),
np.array(sab_pred['sole_1']),
np.array(sab_pred['legal_1']),
np.array(sab_pred['auto_ping_queue_1']),
np.array(sab_pred['requester_type']),
np.array(sab_pred['gender'])]

In [ ]:
kfold_preds = np.zeros((sab_pred.shape[0], len(y.unique())))
n_splits = 5
for i in range(5):
    print('#'*25)
    print('### MODEL %i'%(i+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
    model.load_weights('../input/v55fold/v5-learnedemb-%i.h5'%i)

    print('Predicting Test...')
    preds = model.predict(inputs_t,verbose=True)
    kfold_preds += preds/n_splits

In [ ]:
y_pred = []
for k in range(sab_pred.shape[0]):
    a = np.argmax(kfold_preds[k,])
    y_pred.append(a)

In [ ]:
sab['service_title'].iloc[idx] = y_pred

In [ ]:
sab['service_title'].iloc[idx] = sab['service_title'].iloc[idx].map(labeling_y.set_index('service_title_l').to_dict()['service_title'])

In [ ]:
df2 = pd.read_csv("../input/irkutsk/train_3lags_semibalanced.csv", low_memory=False)

In [ ]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [ ]:
df2 = df2.drop(['service_3', 'service_title_3', 'mfc_3', 'internal_status_3',
       'external_status_3', 'order_type_3', 'department_id_3',
       'custom_service_id_3', 'service_level_3', 'is_subdep_3', 'is_csid_3',
       'month_3', 'week_3', 'year_3', 'dayofweek_3', 'day_part_3', 'person_3',
       'sole_3', 'legal_3', 'auto_ping_queue_3'], axis=1)

In [ ]:
df2 = df2.drop(['proc_time_3', 'win_count_3'], axis=1)

In [ ]:
df2.dropna(inplace=True)

In [ ]:
df_test = df2[['service_title']]

In [ ]:
df_train = df2.drop(['service_title', 'order_count'], axis=1)

In [ ]:
df_train = df_train[['service_1', 'service_title_1', 'mfc_1', 'internal_status_1',
       'external_status_1', 'order_type_1', 'department_id_1',
       'custom_service_id_1', 'service_level_1', 'is_subdep_1', 'is_csid_1',
       'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1', 'person_1',
       'sole_1', 'legal_1', 'auto_ping_queue_1', 'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'order_type_2', 'department_id_2',
       'custom_service_id_2', 'service_level_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2',
       'year_2', 'legal_2', 'auto_ping_queue_2', 'requester_type', 'gender',
       'age']]

In [ ]:
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [ ]:
df_train.drop(['index'], axis=1, inplace=True)
df_test.drop(['index'], axis=1, inplace=True)

In [ ]:
categorical = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1',
       'person_1', 'sole_1', 'legal_1', 'auto_ping_queue_1',
       'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'order_type_2', 'department_id_2',
       'custom_service_id_2', 'service_level_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2', 'year_2',
       'legal_2', 'auto_ping_queue_2','service_3',
       'service_title_3', 'mfc_3', 'internal_status_3', 'external_status_3',
       'order_type_3', 'department_id_3', 'custom_service_id_3',
       'service_level_3', 'is_subdep_3', 'is_csid_3', 'month_3', 'week_3', 'year_3',
       'dayofweek_3', 'day_part_3', 'person_3', 'sole_3', 'legal_3',
       'auto_ping_queue_3',
       'requester_type', 'gender']

In [ ]:
cat = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1',
       'service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'department_id_2', 'month_2', 'week_2', 'year_2',
       'custom_service_id_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'service_3',
       'service_title_3', 'mfc_3', 'internal_status_3', 'external_status_3',
       'department_id_3', 'custom_service_id_3',
       'is_subdep_3', 'is_csid_3', 'month_3', 'week_3', 'year_3',
       'dayofweek_3', 
       'requester_type', 'gender']

In [ ]:
X = df_train
y = df_test['service_title']

In [ ]:
categorical = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1',
       'person_1', 'sole_1', 'legal_1', 'auto_ping_queue_1','service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'order_type_2', 'department_id_2',
       'custom_service_id_2', 'service_level_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2', 'year_2',
       'legal_2', 'auto_ping_queue_2',
       'requester_type', 'gender']

cat = ['service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'service_2', 'service_title_2', 'mfc_2',
       'internal_status_2', 'external_status_2',
       'department_id_2', 'custom_service_id_2', 'month_2', 'week_2', 'year_2',
       'is_subdep_2', 'is_csid_2', 'dayofweek_2', 'requester_type', 'gender']

In [ ]:
X[cat] = X[cat].astype('Int64')
X[cat] = X[cat].astype('object')

In [ ]:
X[categorical] = X[categorical].fillna('NA')

In [ ]:
X = reduce_mem_usage(X)

In [ ]:
X['person_1'] = X['person_1'].astype('int32')
X['sole_1'] = X['sole_1'].astype('int32')
X['legal_1'] = X['legal_1'].astype('int32')
X['auto_ping_queue_1'] = X['auto_ping_queue_1'].astype('int32')

In [ ]:
lag_2 = ['service_2', 'service_title_2', 'mfc_2', 'internal_status_2',
       'external_status_2', 'order_type_2', 'department_id_2',
       'custom_service_id_2', 'service_level_2', 'is_subdep_2', 'is_csid_2',
       'dayofweek_2', 'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2', 'year_2',
       'legal_2', 'auto_ping_queue_2']
X[lag_2] = X[lag_2].astype('str')

In [ ]:
from sklearn import preprocessing
labeling = []
for col in X[categorical].columns:
    d = pd.DataFrame()
    d[col] = X[col].unique()
    le = preprocessing.LabelEncoder()
    le.fit(d[col])
    d[col+'_l'] = le.transform(d[col])
    d = d.sort_values(by=[col+'_l']).reset_index(drop=True)
    labeling.append(d)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X[['age']])
X[['age']] = scaler.transform(X[['age']])

In [ ]:
d = pd.DataFrame()
d['service_title'] = y.unique()
le = preprocessing.LabelEncoder()
le.fit(d['service_title'])
d['service_title'+'_l'] = le.transform(d['service_title'])
d = d.sort_values(by=['service_title'+'_l']).reset_index(drop=True)
labeling_y = d

In [ ]:
i = 0
for col in X[categorical].columns:
    X[col] = X[col].map(labeling[i].set_index(col).to_dict()[col+'_l'])
    i += 1

In [ ]:
y = y.map(labeling_y.set_index('service_title').to_dict()['service_title'+'_l'])

In [ ]:
def build_model():
    service_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_1')
    service_title_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_title_1')
    mfc_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='mfc_1')
    internal_status_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='internal_status_1')
    external_status_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='external_status_1')
    order_type_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='order_type_1')
    department_id_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='department_id_1')
    custom_service_id_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='custom_service_id_1')
    service_level_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_level_1')
    is_subdep_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_subdep_1')
    is_csid_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_csid_1')
    dayofweek_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='dayofweek_1')
    day_part_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='day_part_1')
    month_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='month_1')
    week_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='week_1')
    year_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='year_1')
    person_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='person_1')
    sole_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='sole_1')
    legal_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='legal_1')
    auto_ping_queue_1 = tf.keras.layers.Input((1,), dtype=tf.int32, name='auto_ping_queue_1')
    
    service_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_2')
    service_title_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_title_2')
    mfc_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='mfc_2')
    internal_status_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='internal_status_2')
    external_status_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='external_status_2')
    order_type_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='order_type_2')
    department_id_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='department_id_2')
    custom_service_id_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='custom_service_id_2')
    service_level_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='service_level_2')
    is_subdep_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_subdep_2')
    is_csid_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='is_csid_2')
    dayofweek_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='dayofweek_2')
    day_part_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='day_part_2')
    month_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='month_2')
    week_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='week_2')
    year_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='year_2')
    person_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='person_2')
    sole_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='sole_2')
    legal_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='legal_2')
    auto_ping_queue_2 = tf.keras.layers.Input((1,), dtype=tf.int32, name='auto_ping_queue_2')
    requester_type = tf.keras.layers.Input((1,), dtype=tf.int32, name='requester_type')
    gender = tf.keras.layers.Input((1,), dtype=tf.int32, name='gender')
    age = tf.keras.layers.Input((1,), dtype=tf.float64, name='age')
    
    service_1_embedding = tf.keras.layers.Embedding(len(X['service_1'].unique()), 11, input_length=1, name='service_1_embedding')(service_1)
    service_title_1_embedding = tf.keras.layers.Embedding(len(X['service_title_1'].unique()), 50, input_length=1, name='service_title_1_embedding')(service_title_1)
    mfc_1_embedding = tf.keras.layers.Embedding(len(X['mfc_1'].unique()), 50, input_length=1, name='mfc_1_embedding')(mfc_1)
    internal_status_1_embedding = tf.keras.layers.Embedding(len(X['internal_status_1'].unique()), 5, input_length=1, name='internal_status_1_embedding')(internal_status_1)
    external_status_1_embedding = tf.keras.layers.Embedding(len(X['external_status_1'].unique()), 14, input_length=1, name='external_status_1_embedding')(external_status_1)
    order_type_1_embedding = tf.keras.layers.Embedding(len(X['order_type_1'].unique()), 2, input_length=1, name='order_type_1_embedding')(order_type_1)
    department_id_1_embedding = tf.keras.layers.Embedding(len(X['department_id_1'].unique()), 46, input_length=1, name='department_id_1_embedding')(department_id_1)
    custom_service_id_1_embedding = tf.keras.layers.Embedding(len(X['custom_service_id_1'].unique()), 26, input_length=1, name='custom_service_id_1_embedding')(custom_service_id_1)
    service_level_1_embedding = tf.keras.layers.Embedding(len(X['service_level_1'].unique()), 3, input_length=1, name='service_level_1_embedding')(service_level_1)
    is_subdep_1_embedding = tf.keras.layers.Embedding(len(X['is_subdep_1'].unique()), 1, input_length=1, name='is_subdep_1_embedding')(is_subdep_1)
    is_csid_1_embedding = tf.keras.layers.Embedding(len(X['is_csid_1'].unique()), 1, input_length=1, name='is_csid_1_embedding')(is_csid_1)
    dayofweek_1_embedding = tf.keras.layers.Embedding(len(X['dayofweek_1'].unique()), 4, input_length=1, name='dayofweek_1_embedding')(dayofweek_1)
    day_part_1_embedding = tf.keras.layers.Embedding(len(X['day_part_1'].unique()), 2, input_length=1, name='day_part_1_embedding')(day_part_1)
    month_1_embedding = tf.keras.layers.Embedding(len(X['month_1'].unique()), 6, input_length=1, name='month_1_embedding')(month_1)
    week_1_embedding = tf.keras.layers.Embedding(len(X['week_1'].unique()), 26, input_length=1, name='week_1_embedding')(week_1)
    year_1_embedding = tf.keras.layers.Embedding(len(X['year_1'].unique()), 1, input_length=1, name='year_1_embedding')(year_1)
    person_1_embedding = tf.keras.layers.Embedding(len(X['person_1'].unique()), 1, input_length=1, name='person_1_embedding')(person_1)
    sole_1_embedding = tf.keras.layers.Embedding(len(X['sole_1'].unique()), 1, input_length=1, name='sole_1_embedding')(sole_1)
    legal_1_embedding = tf.keras.layers.Embedding(len(X['legal_1'].unique()), 1, input_length=1, name='legal_1_embedding')(legal_1)
    auto_ping_queue_1_embedding = tf.keras.layers.Embedding(len(X['auto_ping_queue_1'].unique()), 1, input_length=1, name='auto_ping_queue_1_embedding')(auto_ping_queue_1)
    
    service_2_embedding = tf.keras.layers.Embedding(len(X['service_2'].unique()), 11, input_length=1, name='service_2_embedding')(service_2)
    service_title_2_embedding = tf.keras.layers.Embedding(len(X['service_title_2'].unique()), 50, input_length=1, name='service_title_2_embedding')(service_title_2)
    mfc_2_embedding = tf.keras.layers.Embedding(len(X['mfc_2'].unique()), 50, input_length=1, name='mfc_2_embedding')(mfc_2)
    internal_status_2_embedding = tf.keras.layers.Embedding(len(X['internal_status_2'].unique()), 5, input_length=1, name='internal_status_2_embedding')(internal_status_2)
    external_status_2_embedding = tf.keras.layers.Embedding(len(X['external_status_2'].unique()), 14, input_length=1, name='external_status_2_embedding')(external_status_2)
    order_type_2_embedding = tf.keras.layers.Embedding(len(X['order_type_2'].unique()), 2, input_length=1, name='order_type_2_embedding')(order_type_2)
    department_id_2_embedding = tf.keras.layers.Embedding(len(X['department_id_2'].unique()), 46, input_length=1, name='department_id_2_embedding')(department_id_2)
    custom_service_id_2_embedding = tf.keras.layers.Embedding(len(X['custom_service_id_2'].unique()), 26, input_length=1, name='custom_service_id_2_embedding')(custom_service_id_2)
    service_level_2_embedding = tf.keras.layers.Embedding(len(X['service_level_2'].unique()), 3, input_length=1, name='service_level_2_embedding')(service_level_2)
    is_subdep_2_embedding = tf.keras.layers.Embedding(len(X['is_subdep_2'].unique()), 1, input_length=1, name='is_subdep_2_embedding')(is_subdep_2)
    is_csid_2_embedding = tf.keras.layers.Embedding(len(X['is_csid_2'].unique()), 1, input_length=1, name='is_csid_2_embedding')(is_csid_2)
    dayofweek_2_embedding = tf.keras.layers.Embedding(len(X['dayofweek_2'].unique()), 4, input_length=1, name='dayofweek_2_embedding')(dayofweek_2)
    day_part_2_embedding = tf.keras.layers.Embedding(len(X['day_part_2'].unique()), 2, input_length=1, name='day_part_2_embedding')(day_part_2)
    month_2_embedding = tf.keras.layers.Embedding(len(X['month_2'].unique()), 6, input_length=1, name='month_2_embedding')(month_2)
    week_2_embedding = tf.keras.layers.Embedding(len(X['week_2'].unique()), 26, input_length=1, name='week_2_embedding')(week_2)
    year_2_embedding = tf.keras.layers.Embedding(len(X['year_2'].unique()), 1, input_length=1, name='year_2_embedding')(year_2)
    person_2_embedding = tf.keras.layers.Embedding(len(X['person_2'].unique()), 1, input_length=1, name='person_2_embedding')(person_2)
    sole_2_embedding = tf.keras.layers.Embedding(len(X['sole_2'].unique()), 1, input_length=1, name='sole_2_embedding')(sole_2)
    legal_2_embedding = tf.keras.layers.Embedding(len(X['legal_2'].unique()), 1, input_length=1, name='legal_2_embedding')(legal_2)
    auto_ping_queue_2_embedding = tf.keras.layers.Embedding(len(X['auto_ping_queue_2'].unique()), 1, input_length=1, name='auto_ping_queue_2_embedding')(auto_ping_queue_2)
    
    requester_type_embedding = tf.keras.layers.Embedding(len(X['requester_type'].unique()), 2, input_length=1, name='requester_type_embedding')(requester_type)
    gender_embedding = tf.keras.layers.Embedding(len(X['gender'].unique()), 1, input_length=1, name='gender_embedding')(gender)
    age_reshape = tf.keras.layers.Reshape((1, 1), name='age_reshape')(age)

    concatenated = tf.keras.layers.Concatenate()([service_1_embedding, 
                                                  service_title_1_embedding,
                                                 mfc_1_embedding,
                                                 internal_status_1_embedding,
                                                 external_status_1_embedding,
                                                 order_type_1_embedding,
                                                 department_id_1_embedding,
                                                 custom_service_id_1_embedding,
                                                 service_level_1_embedding,
                                                 is_subdep_1_embedding,
                                                 is_csid_1_embedding,
                                                 dayofweek_1_embedding,
                                                 day_part_1_embedding,
                                                 month_1_embedding,
                                                 year_1_embedding,
                                                 person_1_embedding,
                                                 sole_1_embedding,
                                                 legal_1_embedding,
                                                 auto_ping_queue_1_embedding,
                                                 service_2_embedding, 
                                                 service_title_2_embedding,
                                                 mfc_2_embedding,
                                                 internal_status_2_embedding,
                                                 external_status_2_embedding,
                                                 order_type_2_embedding,
                                                 department_id_2_embedding,
                                                 custom_service_id_2_embedding,
                                                 service_level_2_embedding,
                                                 is_subdep_2_embedding,
                                                 is_csid_2_embedding,
                                                 dayofweek_2_embedding,
                                                 day_part_2_embedding,
                                                 month_2_embedding,
                                                 year_2_embedding,
                                                 person_2_embedding,
                                                 sole_2_embedding,
                                                 legal_2_embedding,
                                                 auto_ping_queue_2_embedding,
                                                 requester_type_embedding,
                                                 gender_embedding,
                                                 age_reshape])

    conv_0 = tf.keras.layers.Conv1D(256, 3, padding='same', activation='relu')(concatenated)
    conv_1 = tf.keras.layers.Conv1D(256, 2, padding='same', activation='relu')(concatenated)
    conv_2 = tf.keras.layers.Conv1D(256, 6, padding='same', activation='relu')(concatenated)
    conv_0 = tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu')(conv_0)
    conv_1 = tf.keras.layers.Conv1D(128, 2, padding='same', activation='relu')(conv_1)
    conv_2 = tf.keras.layers.Conv1D(128, 6, padding='same', activation='relu')(conv_2)
    concatenated_tensor = tf.keras.layers.Concatenate(axis=1)([conv_0, conv_1, conv_2])
    out = tf.keras.layers.Flatten()(concatenated_tensor)
    
    out = tf.keras.layers.Dropout(0.1)(out)
    
    out = tf.keras.layers.Dense(len(y.unique()), activation='softmax', name='service_title')(out)

    model = tf.keras.models.Model(inputs=[service_1, 
                                          service_title_1, 
                                          mfc_1,
                                         internal_status_1,
                                         external_status_1,
                                         order_type_1,
                                         department_id_1,
                                         custom_service_id_1,
                                         service_level_1,
                                         is_subdep_1,
                                         is_csid_1,
                                         dayofweek_1,
                                         day_part_1,
                                         month_1,
                                         week_1,
                                         year_1,
                                         person_1,
                                         sole_1,
                                         legal_1,
                                         auto_ping_queue_1,
                                         service_2, 
                                         service_title_2, 
                                         mfc_2,
                                         internal_status_2,
                                         external_status_2,
                                         order_type_2,
                                         department_id_2,
                                         custom_service_id_2,
                                         service_level_2,
                                         is_subdep_2,
                                         is_csid_2,
                                         dayofweek_2,
                                         day_part_2,
                                         month_2,
                                         week_2,
                                         year_2,
                                         person_2,
                                         sole_2,
                                         legal_2,
                                         auto_ping_queue_2,
                                         requester_type,
                                         gender,
                                         age], 
                                  outputs=out)

    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

    return model

In [ ]:
sab_pred = sab[['requester']]

sab_pred = pd.merge(sab_pred, df, how='left', on='requester')

sab_pred.drop(['service_3',
'service_title_3',
'mfc_3',
'internal_status_3',
'external_status_3',
'order_type_3',
'department_id_3',
'custom_service_id_3',
'service_level_3',
'is_subdep_3',
'is_csid_3',
'proc_time_3',
'dayofweek_3',
'day_part_3',
'person_3',
'sole_3',
'legal_3',
'auto_ping_queue_3',
'win_count_3', 'year_3', 'month_3', 'week_3'], axis=1 , inplace=True)

sab_pred.drop(['requester'], axis=1, inplace=True)

In [ ]:
idx = sab_pred.loc[sab_pred['order_count'] > 1].index

In [ ]:
sab_pred = sab_pred.iloc[idx]

In [ ]:
sab_pred.drop(['service_2',
'service_title_2',
'mfc_2',
'internal_status_2',
'external_status_2',
'order_type_2',
'department_id_2',
'custom_service_id_2',
'service_level_2',
'is_subdep_2',
'is_csid_2',
'proc_time_2',
'dayofweek_2',
'day_part_2',
'person_2',
'sole_2',
'legal_2',
'auto_ping_queue_2',
'win_count_2',
'month_2',
'week_2',
'year_2'], axis=1 , inplace=True)

In [ ]:
sab_pred.drop(['proc_time', 'win_count', 'proc_time_1', 'win_count_1', 'order_count'], axis=1, inplace=True)

In [ ]:
sab_pred = sab_pred[['service', 'service_title', 'mfc', 'internal_status',
       'external_status', 'order_type', 'department_id',
       'custom_service_id', 'service_level', 'is_subdep', 'is_csid',
       'dayofweek', 'day_part', 'month', 'week', 'year', 'person',
       'sole', 'legal', 'auto_ping_queue', 'service_1',
       'service_title_1', 'mfc_1', 'internal_status_1', 'external_status_1',
       'order_type_1', 'department_id_1', 'custom_service_id_1',
       'service_level_1', 'is_subdep_1', 'is_csid_1', 'dayofweek_1',
       'day_part_1', 'person_1', 'sole_1', 'month_1', 'week_1', 'year_1',
       'legal_1', 'auto_ping_queue_1', 'requester_type', 'gender', 'age']]

In [ ]:
new_cols=X.columns
sab_pred.rename(columns=dict(zip(sab_pred.columns, new_cols)),inplace=True)
sab_pred[cat] = sab_pred[cat].astype('Int64')
sab_pred[cat] = sab_pred[cat].astype('object')

In [ ]:
sab_pred

In [ ]:
sab_pred['person_1'] = sab_pred['person_1'].map({True: 1, False:0})
sab_pred['sole_1'] = sab_pred['sole_1'].map({True: 1, False:0})
sab_pred['legal_1'] = sab_pred['legal_1'].map({True: 1, False:0})
sab_pred['auto_ping_queue_1'] = sab_pred['auto_ping_queue_1'].map({True: 1, False:0})

In [ ]:
sab_pred['person_2'] = sab_pred['person_2'].map({True: '1.0', False: '0.0'})
sab_pred['sole_2'] = sab_pred['sole_2'].map({True: '1.0', False:'0.0'})
sab_pred['legal_2'] = sab_pred['legal_2'].map({True: '1.0', False:'0.0'})
sab_pred['auto_ping_queue_2'] = sab_pred['auto_ping_queue_2'].map({True: '1.0', False:'0.0'})

In [ ]:
sab_pred[['service_1', 'service_title_1', 'mfc_1', 'internal_status_1',
       'external_status_1', 'order_type_1', 'department_id_1',
       'custom_service_id_1', 'service_level_1', 'is_subdep_1', 'is_csid_1',
       'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1', 'person_1',
       'sole_1', 'legal_1', 'auto_ping_queue_1']] = sab_pred[['service_1', 'service_title_1', 'mfc_1', 'internal_status_1',
       'external_status_1', 'order_type_1', 'department_id_1',
       'custom_service_id_1', 'service_level_1', 'is_subdep_1', 'is_csid_1',
       'dayofweek_1', 'day_part_1', 'month_1', 'week_1', 'year_1', 'person_1',
       'sole_1', 'legal_1', 'auto_ping_queue_1']].fillna(sab_pred.mode().iloc[0])

In [ ]:
sab_pred[['service_2',
       'service_title_2', 'mfc_2', 'internal_status_2', 'external_status_2',
       'order_type_2', 'department_id_2', 'custom_service_id_2',
       'service_level_2', 'is_subdep_2', 'is_csid_2', 'dayofweek_2',
       'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2', 'year_2',
       'legal_2', 'auto_ping_queue_2']] = sab_pred[['service_2',
       'service_title_2', 'mfc_2', 'internal_status_2', 'external_status_2',
       'order_type_2', 'department_id_2', 'custom_service_id_2',
       'service_level_2', 'is_subdep_2', 'is_csid_2', 'dayofweek_2',
       'day_part_2', 'person_2', 'sole_2', 'month_2', 'week_2', 'year_2',
       'legal_2', 'auto_ping_queue_2']].fillna('NA')

In [ ]:
sab_pred['person_1'] = sab_pred['person_1'].astype('int64')
sab_pred['sole_1'] = sab_pred['sole_1'].astype('int64')
sab_pred['legal_1'] = sab_pred['legal_1'].astype('int64')
sab_pred['auto_ping_queue_1'] = sab_pred['auto_ping_queue_1'].astype('int64')

In [ ]:
sab_pred[lag_2] = sab_pred[lag_2].astype('str')

In [ ]:
i = 0
for col in sab_pred[categorical].columns:
    sab_pred[col] = sab_pred[col].map(labeling[i].set_index(col).to_dict()[col+'_l'])
    i += 1

In [ ]:
sab_pred.fillna(sab_pred.mode().iloc[0], inplace=True)

In [ ]:
toint = ['service_1', 'service_title_1', 'mfc_1', 'order_type_1', 'person_1', 'sole_1', 'auto_ping_queue_1', 'day_part_1',
       'internal_status_1', 'external_status_1', 'service_level_1', 'legal_1',
       'department_id_1', 'custom_service_id_1', 'month_1', 'week_1', 'year_1',
       'is_subdep_1', 'is_csid_1', 'dayofweek_1', 'service_title_2',
       'requester_type', 'gender']

In [ ]:
sab_pred[toint] = sab_pred[toint].astype('int64')

In [ ]:
sab_pred['order_type_1'] = sab_pred['order_type_1'].astype('category')
sab_pred['service_level_1'] = sab_pred['service_level_1'].astype('category')
sab_pred['day_part_1'] = sab_pred['day_part_1'].astype('category')

In [ ]:
sab_pred[toint] = sab_pred[toint].astype('int64')

In [ ]:
sab_pred['service_level_2'] = sab_pred['service_level_2'].astype('int64')

In [ ]:
sab_pred[['order_type_1', 'service_level_1', 'day_part_1']] = sab_pred[['order_type_1', 'service_level_1', 'day_part_1']].astype('category')

In [ ]:
sab_pred[['age']] = scaler.transform(sab_pred[['age']])

In [ ]:
inputs_t = [np.array(sab_pred['service_1']),
np.array(sab_pred['service_title_1']),
np.array(sab_pred['mfc_1']),
np.array(sab_pred['internal_status_1']),
np.array(sab_pred['external_status_1']),
np.array(sab_pred['order_type_1']),
np.array(sab_pred['department_id_1']),
np.array(sab_pred['custom_service_id_1']),
np.array(sab_pred['service_level_1']),
np.array(sab_pred['is_subdep_1']),
np.array(sab_pred['is_csid_1']),
np.array(sab_pred['dayofweek_1']),
np.array(sab_pred['day_part_1']),
np.array(sab_pred['month_1']),
np.array(sab_pred['week_1']),
np.array(sab_pred['year_1']),
np.array(sab_pred['person_1']),
np.array(sab_pred['sole_1']),
np.array(sab_pred['legal_1']),
np.array(sab_pred['auto_ping_queue_1']),
np.array(sab_pred['service_2']),
np.array(sab_pred['service_title_2']),
np.array(sab_pred['mfc_2']),
np.array(sab_pred['internal_status_2']),
np.array(sab_pred['external_status_2']),
np.array(sab_pred['order_type_2']),
np.array(sab_pred['department_id_2']),
np.array(sab_pred['custom_service_id_2']),
np.array(sab_pred['service_level_2']),
np.array(sab_pred['is_subdep_2']),
np.array(sab_pred['is_csid_2']),
np.array(sab_pred['dayofweek_2']),
np.array(sab_pred['day_part_2']),
np.array(sab_pred['month_2']),
np.array(sab_pred['week_2']),
np.array(sab_pred['year_2']),
np.array(sab_pred['person_2']),
np.array(sab_pred['sole_2']),
np.array(sab_pred['legal_2']),
np.array(sab_pred['auto_ping_queue_2']),
np.array(sab_pred['requester_type']),
np.array(sab_pred['gender']),
np.array(sab_pred['age'])]

In [ ]:
kfold_preds = np.zeros((sab_pred.shape[0], labeling_y.shape[0]))
n_splits = 5
for i in range(5):
    print('#'*25)
    print('### MODEL %i'%(i+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
    model.load_weights('../input/categorical-embedding-v4-2lags/v11-learnedemb-%i.h5'%i)

    print('Predicting Test...')
    preds = model.predict(inputs_t,verbose=True)
    kfold_preds += preds/n_splits

In [ ]:
y_pred = []
for k in range(sab_pred.shape[0]):
    a = np.argmax(kfold_preds[k,])
    y_pred.append(a)

In [ ]:
sab['service_title'].iloc[idx] = y_pred

In [ ]:
sab['service_title'].iloc[idx] = sab['service_title'].iloc[idx].map(labeling_y.set_index('service_title_l').to_dict()['service_title'])

In [ ]:
sab['service_title'].value_counts(normalize=True)

In [ ]:
sab.to_csv('nn_5fold_2models_0.csv', index=False)